# Search Engine

In [1]:
import nltk
nltk.download('stopwords');
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from collections import Counter
import pandas as pd
import json
from numpy import linalg as la
import matplotlib.pyplot as plt
import ast
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wenjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## access token informations 
access_token1 = "1240651902712516613-BF3CepSxPUoMr3lDkptNt08f2GwaGZ"
access_token_secret1 = "79ga9EksEgtgpHGJhXy7A4ujb31pw1AjwP8lOlaYtmDdk"

consumer_key1 = "ieusj8Wof83kmsVe9PDLJQqn3"
consumer_secret1 = "EpZYsMfEqvHSwFQyAfz2wCdVujPTtr7JQtvkBt1g9sMg5t0m3B"

#Generate API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

###### 1. Load data from json

In [ ]:
#load data from output.json scrapped from twitter
with open("../other-outputs/output.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

df_tweets = pd.DataFrame.from_records(data)

# Get unique tweets
tweets=df_tweets[df_tweets["retweeted_status"].isna()]
tweets.head()

##### 2. Print information 

In [ ]:
def print_retweets_uniqueTweets_uniqueUsers(df_tweets):
    retweets_ = df_tweets["retweeted_status"].apply(lambda x: 0 if str(x) == "nan" else 1)
    tot_retweets = sum(retweets_)
    unique_tweets = len(retweets_) - tot_retweets
    
    users_ = df_tweets.user.apply(lambda x: x["id"])
    
    tot_users = len(set(users_))
    
    print("Total Retweets:", tot_retweets)

    print("Unique Tweets:", unique_tweets)
    
    print("Unique Users:", tot_users)

In [ ]:
print_retweets_uniqueTweets_uniqueUsers(df_tweets)

In [ ]:
len(df_tweets)

##### 3. Update (Don't execute this blog)

In [ ]:
def update(tweets):
    """
    
    
    Update number of the tweet's likes and retweed.
    """
    for index, tweet in tweets.iterrows():
        try:
            tweet = api.get_status(tweet["id"])
            tweets.loc[index, "favorite_count"] = tweet.favorite_count
            tweets.loc[index, "retweet_count"] = tweet.retweet_count
        except:
            pass


In [ ]:
update(tweets)


In [ ]:
tweets["favorite_count"].head()

In [ ]:
tweets.to_csv('../other-outputs/tweets.tsv', sep = '\t', index=False)

#####  4. Load data from TSV

In [3]:
tweets = pd.read_csv("../other-outputs/tweets.tsv", sep='\t', index_col=0)


###### 5. Consult the more likes and more retweets.

In [4]:
more_likes = 0
text_with_more_likes = ""
for index, tweet in tweets.iterrows():
    likes_tweet = tweet["favorite_count"]
    if likes_tweet > more_likes:
        more_likes = likes_tweet
        text_with_more_likes =  tweet["text"]
print("Most liked tweet: {}, with a total of {} likes.".format(text_with_more_likes, more_likes))

Most liked tweet: When a former VP of Pfizer is calling for the immediate suspension of all SARS CoV 2 vaccine studies, because of th… https://t.co/s6yvRBZKoh, with a total of 171 likes.


In [5]:
more_retweets = 0
text_with_more_retweets = ""
for index, tweet in tweets.iterrows():
    num_retweets = tweet["retweet_count"]
    if num_retweets > more_retweets:
        more_retweets = num_retweets
        text_with_more_retweets = tweet["text"]
        
print("Most retweets tweet: {}, with a total of {} retweets.".format(text_with_more_retweets, more_retweets))

Most retweets tweet: When a former VP of Pfizer is calling for the immediate suspension of all SARS CoV 2 vaccine studies, because of th… https://t.co/s6yvRBZKoh, with a total of 104 retweets.


##### 6. Pre-processing the Tweets

In [6]:
def getTerms(tweet):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    tweet -- string (text) to be preprocessed
    
    Returns:
    tweet - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    tweet =  tweet.lower() ## Transform in lowercase
    tweet = tweet.replace("#", "") ## removing the “#” from the word 
    tweet =  tweet.split(" ") ## Tokenize the text to get a list of terms
    tweet = [word for word in tweet if word not in stops] #Remove stopwords. 
    tweet =[stemming.stem(word) for word in tweet] 
    return tweet

##### 7. Inverted-index 

In [7]:
def create_index_tfidf(tweets, numTweets):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numTweets -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    index=defaultdict(list) 
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    IdIndex = {} # dictionary to map page titles to page ids
    idf=defaultdict(float) # dictionary with inversed document frequency 
    
    for i,tweet in tweets.iterrows():
        tweet_id = i
        terms = getTerms(tweet["text"]) #page_title + page_text
        userid = int(tweet["id"])            
        IdIndex[tweet_id]=userid  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)
        
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]+= 1  # increment df for current term   
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
         # Compute idf
        for term in df:
            idf[term] = np.round(np.log(float(numTweets/df[term])),4)                   
                    
    return index, tf, df, idf, IdIndex

In [8]:
#Generate Index and idIndex. 
start_time = time.time()
numTweets = len(tweets)
index, tf, df, idf, idIndex = create_index_tfidf(tweets, numTweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))


Total time to create the index: 43.68 seconds


#### 8. output search 

In [10]:
def output_query(ranked_docs): 
    """
        This functin is used to print the result. 
    """
    top = 20

    print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
    resultTmp = []

    for d_id in ranked_docs[:top] :
        ID = idIndex[d_id]
        #print("page_id= {} - page_title: {}".format(d_id, titleIndex[d_id]))
        tweetTmp = tweets[tweets["id"] == ID]
        tweet = tweetTmp["text"].values[0]
        username = ast.literal_eval(tweetTmp["user"].values[0])["screen_name"]
        date = tweetTmp["created_at"].values[0]
        hashtags_lists = ast.literal_eval(tweetTmp["entities"].values[0])["hashtags"]
        hashtags = [tag["text"] for tag in hashtags_lists]
        likes = tweetTmp["favorite_count"].values[0]
        retweets = tweetTmp["retweet_count"].values[0]
        url = ast.literal_eval(tweetTmp["user"].values[0])["url"]

        resultTmp.append([tweet,username,date,hashtags,likes,retweets,url])

    result = pd.DataFrame(resultTmp,columns=["Tweet", "Username", "Date", "Hashtags", "Likes", "Retweets", "Url"])
    return result

##### 9. Ranking Score

In [11]:
def rankDocuments(terms, docs, index, idf, tf, idIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    idIndex -- mapping between page id and tweet id
    
    Returns:
    Print the list of ranked documents
    """
    docVectors=defaultdict(lambda: [0]*len(terms)) 
    queryVector=[0]*len(terms)    
                                                                                                                                                                                                                                                                     
    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})    
    query_norm = la.norm(list(query_terms_count.values()))
    
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            if doc in docs:
                tweetInfo = tweets[tweets["id"] == idIndex[doc]]
                like =  tweetInfo["favorite_count"].values
                retweet = tweetInfo["retweet_count"].values
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:    
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ idIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

##### 10. Search with tf-idf ranking

In [12]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, idIndex)   
    return ranked_docs

In [17]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    

output_query(ranked_docs)

Insert your query:

covid-19 america

Top 20 results out of 155 for the seached query:



,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,Lock them all up they are responsible for the ...,madog190157,Sat Dec 05 10:13:18 +0000 2020,[],1,0,None
1,@WillyMasterson @JoeBiden Everyone in America ...,Johnsontb1,Sat Dec 05 10:22:01 +0000 2020,[],0,0,None
2,Reading Covid discourse from America on Twitte...,JamieCPitman,Sat Dec 05 12:29:49 +0000 2020,[],0,0,http://Instagram.com/jcpitman
3,@chris_herd I’m witnessing it now. My concern ...,NakoMbelle,Sat Dec 05 10:21:15 +0000 2020,[],1,0,http://www.fintechrecruiters.com
4,After shed loads of information warnings &amp;...,alanjoh1949,Sat Dec 05 10:11:10 +0000 2020,[],4,1,None
5,@lack__lustre Covid-19,RanaTal80613033,Sat Dec 05 10:21:33 +0000 2020,[],1,0,None
6,What have you experienced with this Covid-19 p...,misstmoraa,Sat Dec 05 10:20:54 +0000 2020,[],0,0,None
7,As well did COVID-19 Deaths https://t.co/kX8Hw...,ParodyTheodore,Sat Dec 05 10:22:04 +0000 2020,[],3,0,None
8,"Have Covid-19 passport, will travel https://t....",BusinessTimes,Sat Dec 05 10:21:13 +0000 2020,[],0,0,None
9,COVID-19 Malaysia Updates https://t.co/9aSK7l1lYz,HowKokKeng1,Sat Dec 05 10:13:18 +0000 2020,[],0,0,None


##### 11. My score Ranking

In [40]:
def rankDocumentsWithLikes(terms, docs, index, idf, tf, idIndex):
    """
    In this function we compute the out ranking score with like and retweets counts. 
    Because in this case, the amount of rwteets and the number of user's likes
    are also one of the important factors for ranking. 
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    idIndex -- mapping between page id and tweet id
    
    Returns:
    Print the list of ranked documents
    """
    docVectors=defaultdict(lambda: [0]*len(terms)) 
    queryVector=[0]*len(terms)    
                                                                                                                                                                                                                                                                     
    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})    
    query_norm = la.norm(list(query_terms_count.values()))
    
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            if doc in docs:
                tweetInfo = tweets[tweets["id"] == idIndex[doc]]
                like =  tweetInfo["favorite_count"].values
                retweet = tweetInfo["retweet_count"].values
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term] + (like + retweet)

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:    
    docScores=[ [np.dot(np.squeeze(curDocVec), np.squeeze(queryVector)), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ idIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

###### 11. My-Score + cosine similarity

In [41]:
def my_score(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocumentsWithLikes(query, docs, index, idf, tf, idIndex)   
    return ranked_docs

In [58]:
print("Insert your query:\n")
query = input()
ranked_docs = my_score(query, index)    
output_query(ranked_docs)

Insert your query:

coronavirus death rate barcelona

Top 20 results out of 122 for the seached query:



,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,"Which part? \n\nThe virus, which is part of th...",mynameisjerm,Sat Dec 05 12:29:46 +0000 2020,[],75,7,https://jermwarfare.com
1,"The man behind Sweden’s coronavirus strategy, ...",business,Sat Dec 05 10:20:05 +0000 2020,[],40,13,http://www.bloomberg.com
2,@MichaelYeadon3 Please share with everyone\n\n...,lyne_ian,Sat Dec 05 10:20:22 +0000 2020,[],21,8,None
3,Whenever this coronavirus shit ends I’m going ...,bloodyfaceHS,Sat Dec 05 10:12:13 +0000 2020,[],24,0,http://twitch.tv/bloodyfacehs
4,@GerardBattenUK Please share with everyone\n\n...,lyne_ian,Sat Dec 05 10:21:41 +0000 2020,[],11,11,None
5,MoscowMitch has been the cause of thousands of...,RoyHay7,Sat Dec 05 10:20:10 +0000 2020,[],10,4,None
6,"Pakistan reports 44 deaths by Covid-19 and 3,1...",LeAnam__,Sat Dec 05 10:21:04 +0000 2020,[COVIDSecondWave],14,0,http://Instagram.com/anamblogga_/
7,Health minister tests positive after getting c...,NigeriaNewsdesk,Sat Dec 05 10:22:04 +0000 2020,[],7,10,None
8,UK hit by worst economic hit then anyone else ...,mmaher70,Sat Dec 05 10:20:55 +0000 2020,[coronavirus],4,3,None
9,"Last week, Dr. Jonathan Reiner of George Washi...",WoodwarddianneJ,Sat Dec 05 10:12:23 +0000 2020,[],3,2,None


##### Wordcloud

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
import re

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))

In [ ]:
bag_of_words = {}
for text in df_tweets["text"]:
    
    # remove "RT" string indicating a retweet
    text = text.replace("RT ", "").strip()
    
    # lowering text
    text = text.lower()
    
    # removing all the punctuations
    text = re.sub(r'[^\w\s]','',text).strip()
    
    # tokenize the text
    lst_text = text.split()
    
    # remove stopwords
    lst_text = [x for x in lst_text if x not in STOPWORDS]
    
        
    # create bag-of-words - for each word the frequency of the word in the corpus
    for w in lst_text:
        if w not in bag_of_words:
            bag_of_words[w] = 0
        bag_of_words[w] +=1

In [ ]:
def plot_wordcloud(title, dic_):
    fig, ax = plt.subplots(1, 1, figsize=(18,7))
    wordcloud = WordCloud(background_color="white",width=1600, height=800)
    wordcloud = wordcloud.generate_from_frequencies(dic_)
    ax.axis("off")     
    ax.imshow(wordcloud, interpolation='bilinear')

    ax.set_title(title)
    plt.tight_layout()
    fig.subplots_adjust(top=0.8)
    plt.show()

In [ ]:
plot_wordcloud("WordCloud - All Tweets", bag_of_words)

#### Bar plot of the 10 most frequent words

In [ ]:
def extract_hashtags():
    """
    
    """
    
    hashtags_lists = df_tweets["entities"].apply(lambda x: x["hashtags"])
    
    hashtags = hashtags_lists.apply(lambda x: x[0]["text"].lower() if x != [] else None)
    
    hashtags_by_frequency = Counter(hashtags)
    hashtags_by_frequency = {k: hashtags_by_frequency[k] for k in hashtags_by_frequency if k != None} 
    
    return hashtags_by_frequency

In [ ]:
hashtags_by_frequency = extract_hashtags()
hashtags_by_frequency

In [ ]:
df_hashtags = pd.DataFrame(hashtags_by_frequency.items())
df_hashtags.columns = ["hashtag", "count"]
df_hashtags.set_index("hashtag", inplace=True)
df_hashtags.sort_values("count", inplace=True, ascending=False)

In [ ]:
df_hashtags.head(10).plot.barh()
plt.show()

### Word2vec

In [ ]:
def get_tokens(docs, preprocess=preprocess_string, verbose=10000):
    
    for i, doc in enumerate(docs):
        yield preprocess(doc)#  preprocess
        
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print(f"Progress: {i + 1}")
    return docs